# Scaling XGBoost with Dask and Coiled

> This notebook uses the `xgboost-environment.yml` environment located in the `envs` directory [here](https://github.com/coiled/coiled-resources/tree/main/envs).

This notebook shows you how to solve the common **MemoryError** issue that is thrown whenever you try to train an XGBoost model that doesn't fit into your memory. 

You'll learn how to leverage **distributed [XGBoost](https://xgboost.readthedocs.io/en/latest/) training** for effective modelling on datasets that exceed the hardware limitations of your local machine.

Specifically, you will write code to:
1. Train a distributed XGBoost model locally on a small dataset using [Dask](https://dask.org/), 
2. Scale your distributed XGBoost model to the cloud using Dask and [Coiled](https://coiled.io/) to train on a larger-than-memory dataset,
3. Speed up your training with Pro tips from the Dask core team.

### About the Dataset
We'll be using **a 100GB dataset** containing synthetic data, generated using the `dask-ml make_regression` API. The dataset is stored in the public `coiled-datasets` S3 bucket.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import logging
logger = logging.getLogger("distributed.utils_perf")
logger.setLevel(logging.ERROR)

## 1. Local Distributed XGBoost Model using Dask

By default, XGBoost trains models sequentially. This is fine for smaller projects, but when the size of your dataset and/or ML model exceeds the limitations of your local machine, you will want to leverage the potential of distributed computing.

Starting from version 1.0, XGBoost comes with a native Dask integration that makes this possible. 

It only requires two changes to your regular XGBoostcode:
1. substitute `dtrain = xgb.DMatrix(X_train, y_train)` with `dtrain = xgb.dask.DaskDMatrix(X_train, y_train)`, and
2. substitute `xgb.train(params, dtrain, ...)` with `xgb.dask.train(client, params, dtrain, ...)`

Let's see this in action.

### Instantiate Dask Cluster

We'll begin by instantiating a local version of the Dask distributed scheduler, which will orchestrate the distributed training of our model. Read more about the Dask schedulers [here](https://distributed.dask.org/en/latest/).

In [2]:
from dask.distributed import Client, LocalCluster

# local dask cluster
cluster = LocalCluster(n_workers=4)
client = Client(cluster)
client

2022-07-06 12:46:58,928 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/rpelgrim/Documents/git/coiled-resources/xgboost-with-coiled/dask-worker-space/worker-19u7q0va', purging
2022-07-06 12:46:58,929 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/rpelgrim/Documents/git/coiled-resources/xgboost-with-coiled/dask-worker-space/worker-eitabnb2', purging
2022-07-06 12:46:58,929 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/rpelgrim/Documents/git/coiled-resources/xgboost-with-coiled/dask-worker-space/worker-_uemfips', purging
2022-07-06 12:46:58,929 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/rpelgrim/Documents/git/coiled-resources/xgboost-with-coiled/dask-worker-space/worker-wwajh32z', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52861,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:52877,Total threads: 2
Dashboard: http://127.0.0.1:52880/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:52864,


### Import the Data
Since we are working with a synthetic dataset, we can import the data and start training right away. No preprocessing needed.

For an example notebook with real-world data that does include some preprocessing work, check out [this notebook](https://github.com/coiled/coiled-resources/blob/main/xgboost-with-coiled/coiled-xgboost-arcos-20GB.ipynb) that trains an XGBoost model on a 20GB subset of the ARCOS dataset.

In [3]:
import dask.dataframe as dd

# download data from S3
data = dd.read_parquet(
    "s3://coiled-datasets/synthetic-data/synth-reg-104GB.parquet/", 
    compression="lz4",
    storage_options={"anon": True, 'use_ssl': True},
)

In [4]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,target
npartitions=2750,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Since we're working locally to begin with, we won't be able to process the entire 100GB dataset.

We'll subset the first 10 partitions and persist them to our Dask cluster memory for quicker access.

In [5]:
# select the first 10 partitions
data_local = data.partitions[0:10]
data_local = data_local.persist()

In [6]:
# inspect the first 5 entries
data_local.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,target
0,-1.083516,0.173372,-0.973546,-1.465443,1.973955,-0.922526,1.058072,0.302878,1.160762,-0.690999,...,0.478698,-1.286906,0.037474,-0.448159,-0.652509,-1.205982,0.166634,2.526275,-0.890744,223.602485
1,2.077819,-0.507675,1.188347,-0.958974,0.666332,0.699718,0.416365,-0.006916,-0.561665,-0.535323,...,-0.406144,-0.122424,1.623143,0.438106,-1.510411,-0.909098,-0.416044,0.169660,-1.343285,-63.876627
2,-1.545396,-1.001309,-0.185548,-0.507883,1.223005,0.405486,-0.838138,-0.521867,1.164290,0.566665,...,1.341402,-0.206474,-1.203585,0.796500,-2.083753,0.670345,1.243194,-0.513658,-1.388109,182.856379
3,-0.548436,-0.754629,1.628490,0.954295,0.190117,-0.359459,1.901831,-0.137075,-0.005027,0.918249,...,1.214883,-0.115838,0.287735,-0.115192,-0.499330,0.349165,-1.618127,1.421938,-0.439240,-211.527657
4,-0.981102,0.993449,-0.173022,0.503123,0.823864,0.083351,0.242027,0.661806,0.463781,-0.799858,...,-0.988890,-0.541225,-0.298992,0.306095,0.351885,2.269911,0.465673,0.909917,0.513545,-165.464021


This is looking good.

### Train-Test Splits

The next step is to define our train and test splits. The target feature in this synthetic dataset is the last column, conveniently named "target".

In [7]:
from dask_ml.model_selection import train_test_split

In [8]:
# Create the train-test split
X, y = data_local.iloc[:, :-1], data_local["target"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, shuffle=True, random_state=21
)

### Train XGBoost Model

Now we're all set to start training our XGBoost model.

First, we'll create the XGBoost DMatrix and set the model parameters. We'll use the default parameters for this example.

For more information on training XGBoost models and setting model parameter, have a look at the [XGBoost documentation](https://xgboost.readthedocs.io/en/latest/get_started.html).

In [9]:
import xgboost as xgb

In [10]:
# Create the XGBoost DMatrix for our training and testing splits
dtrain = xgb.dask.DaskDMatrix(client, X_train, y_train)
dtest = xgb.dask.DaskDMatrix(client, X_test, y_test)

# Set model parameters (XGBoost defaults)
params = {
    "max_depth": 6,
    "gamma": 0,
    "eta": 0.3,
    "min_child_weight": 30,
    "objective": "reg:squarederror",
    "grow_policy": "depthwise"
}

Then let's go ahead and train the model.

In [11]:
%%time 
# train the model
output = xgb.dask.train(
    client, params, dtrain, num_boost_round=5,
    evals=[(dtrain, 'train')]
)

[12:50:02] task [xgboost.dask]:tcp://127.0.0.1:52877 got new rank 0
[12:50:02] task [xgboost.dask]:tcp://127.0.0.1:52882 got new rank 1
[12:50:02] task [xgboost.dask]:tcp://127.0.0.1:52883 got new rank 2
[12:50:02] task [xgboost.dask]:tcp://127.0.0.1:52876 got new rank 3


[0]	train-rmse:191.08902
[1]	train-rmse:166.81632
[2]	train-rmse:147.24834
[3]	train-rmse:132.46113
[4]	train-rmse:119.38073
CPU times: user 73.3 ms, sys: 25.4 ms, total: 98.7 ms
Wall time: 1.71 s


And use our trained model together with our testing split to make predictions.

In [12]:
# make predictions
y_pred = xgb.dask.predict(client, output, dtest)

And finally, let's evaluate our results by getting the accuracy score.

In [13]:
from sklearn.metrics import mean_absolute_error

In [14]:
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 93.9991012601532


### Try Locally with Entire Dataset... if you dare...

Unless you're running this on a supercomputer, uncommenting and running the cell below will likely not complete.

But don't just take our word for it, of course ;)

In [ ]:
# # Create the train-test split
# X, y = data.iloc[:, :-1], data["target"]
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.3, shuffle=True, random_state=2
# )

# # Create DaskDMatrices
# dtrain = xgb.dask.DaskDMatrix(client, X_train, y_train)
# dtest = xgb.dask.DaskDMatrix(client, X_test, y_test)

```MemoryError
distributed.batched - ERROR - Error in batched write
```
```
MemoryError
```
```
distributed.worker - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 1.49 GiB -- Worker memory limit: 1.86 GiB
```

## 2. Distributed XGBoost in the Cloud using Dask and Coiled

Let's now expand this workflow to process the entire dataset (>100 GB). 

We'll run the exact same code as above except for **2 changes**:
1. We'll connect Dask to a Coiled cluster in the cloud, instead of to our local CPU cores,
2. We'll work with the entire 100GB dataset, instead of the first 10 partitions.

In the section below we've copied and pasted the cells from above so that you can run this notebook from top to bottom in one go. Alternatively, you could run the cell below (where we instantiate the Coiled Cluster) and then simply re-run the cells above -- making sure to work with the entire dataset, of course.

### Instantiate Coiled Cluster
Let's create our Coiled cluster in the cloud. 

We'll specify a cluster of 50 workers, with 4 CPU cores and 16GB of RAM each. That will allow the entire dataset to fit into the cluster's memory comfortably and should make for quick training.

> *Note: if you're running this using the Coiled Free Tier, you'll want to reduce your **n_workers** to 25 to stay within the Total Core limit.*

In [15]:
import coiled

cluster = coiled.Cluster(
    name="xgboost",
    software="coiled-examples/xgboost-coiled",
    n_workers=50,
    worker_memory='16Gib',
    shutdown_on_close=False,
)

Output()

In [16]:
# connect Dask to your remote cluster
from distributed import Client
client = Client(cluster)

<Client: 'tls://10.4.9.112:8786' processes=26 threads=104, memory=402.73 GiB>

### Inspecting Entire Dataset

Let's load the entire dataset into our Dask dataframe **data**.

As you can see below, it consists of 2750 partitions.

In [17]:
import dask.dataframe as dd

In [18]:
data = dd.read_parquet(
    "s3://coiled-datasets/synthetic-data/synth-reg-104GB.parquet/", 
    compression="lz4",
    storage_options={"anon": True, 'use_ssl': True},
)


In [19]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,target
0,-1.083516,0.173372,-0.973546,-1.465443,1.973955,-0.922526,1.058072,0.302878,1.160762,-0.690999,...,0.478698,-1.286906,0.037474,-0.448159,-0.652509,-1.205982,0.166634,2.526275,-0.890744,223.602485
1,2.077819,-0.507675,1.188347,-0.958974,0.666332,0.699718,0.416365,-0.006916,-0.561665,-0.535323,...,-0.406144,-0.122424,1.623143,0.438106,-1.510411,-0.909098,-0.416044,0.169660,-1.343285,-63.876627
2,-1.545396,-1.001309,-0.185548,-0.507883,1.223005,0.405486,-0.838138,-0.521867,1.164290,0.566665,...,1.341402,-0.206474,-1.203585,0.796500,-2.083753,0.670345,1.243194,-0.513658,-1.388109,182.856379
3,-0.548436,-0.754629,1.628490,0.954295,0.190117,-0.359459,1.901831,-0.137075,-0.005027,0.918249,...,1.214883,-0.115838,0.287735,-0.115192,-0.499330,0.349165,-1.618127,1.421938,-0.439240,-211.527657
4,-0.981102,0.993449,-0.173022,0.503123,0.823864,0.083351,0.242027,0.661806,0.463781,-0.799858,...,-0.988890,-0.541225,-0.298992,0.306095,0.351885,2.269911,0.465673,0.909917,0.513545,-165.464021


In [20]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,target
npartitions=2750,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


### Train / Test Splits

Below we apply the same code we used above to create out training and testing splits. 

We also persist the splits to the cluster's memory for faster training.

In [21]:
from dask_ml.model_selection import train_test_split

In [22]:
# Create the train-test split
X, y = data.iloc[:, :-1], data["target"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, shuffle=True, random_state=13
)

# persist the train/test splits to cluster memory to speed up training
import dask
dask.persist(X_train, X_test, y_train, y_test)

(Dask DataFrame Structure:
                         0        1        2        3        4        5        6        7        8        9       10       11       12       13       14       15       16       17       18       19       20       21       22       23       24       25       26       27       28       29       30       31       32       33       34       35       36       37       38       39       40       41       42       43       44       45       46       47       48       49
 npartitions=2750                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
                   float64  float64  

### XGBoost Training
Alright, the moment we've all been waiting for!

You're now all set to train your distributed XGBoost model on the entire 500GB dataset.

The cells below will create the DaskDMatrix, set the model parameters (using the XGBoost defaults for now) and train your XGBoost model.

In [23]:
import xgboost as xgb

In [24]:
%%time
# Create the XGBoost DMatrices
dtrain = xgb.dask.DaskDMatrix(client, X_train, y_train)
dtest = xgb.dask.DaskDMatrix(client, X_test, y_test)

CPU times: user 7.56 s, sys: 1.75 s, total: 9.32 s
Wall time: 1min 11s


In [25]:
# Set model parameters (XGBoost defaults)
params = {
    "max_depth": 6,
    "gamma": 0,
    "eta": 0.3,
    "min_child_weight": 30,
    "objective": "reg:squarederror",
    "grow_policy": "depthwise"
}

In [26]:
%%time 
# train the model 
output = xgb.dask.train(
    client, params, dtrain, num_boost_round=5,
    evals=[(dtrain, 'train')]
)

CPU times: user 3.39 s, sys: 1.55 s, total: 4.94 s
Wall time: 1min 9s


In [27]:
%%time
# make predictions
y_pred = xgb.dask.predict(client, output, dtest)

CPU times: user 3.59 s, sys: 685 ms, total: 4.28 s
Wall time: 12.6 s


Great work! You just trained an XGBoost model on 100GB of data in a matter of minutes!

### Shutting down the cluster
After our training is done, we can close down the cluster, releasing the resources. Should you forget to do so for whatever reason, Coiled automatically shuts down clusters after 20 minutes of inactivity, to help avoid unnecessary costs.


In [28]:
# Shut down the cluster
client.close()

## 3. Pro Tips to Speed Up Training
Below we’ve collected some pro tips straight from the Dask core team to help you speed up your XGBoost training:

- Increase the number of workers in your Coiled cluster using the `n_workers` keyword argument.
- Re-cast numerical columns to less memory-intensive dtypes. For example, convert float64 into int16 whenever possible. This will reduce the memory load of your dataframe and thereby speed up training.
- The Dask Dashboard is a great way to spot bottle-necks and identify opportunities for increased performance in your code. Watch the initial author of Dask, Matt Rocklin, explain how to get the most out of the Dask Dashboard [here](https://www.youtube.com/watch?v=N_GqzcuGLCY).
- Read Matthew Power’s blog on setting up the Dask Dashboard in your Jupyter Lab environment [here](https://coiled.io/blog/dask-jupyterlab-workflow/). 
- Read Dask core contributor Guido Imperiale’s blog on how to tackle the specific issue of unmanaged memory in Dask workers [here](https://coiled.io/blog/tackling-unmanaged-memory-with-dask/). 



## 4. Recap

Let’s recap what we’ve discussed in this notebook:
- When training XGBoost with large datasets, running out of local memory can be a challenge. 
- Connecting XGboost to a local Dask cluster allows you to make the most out of the multiple cores in your machine.
- If that’s still not enough, you can connect Dask to Coiled and burst to the cloud as and when needed.
- You can tweak your distributed XGBoost performance by inspecting the Dask Dashboard.

We’d love to see you apply distributed XGBoost to a dataset that’s meaningful to you. If you’d like to try, swap your dataset into this notebook and see how well it does! 

Let us know how you get on in our [Coiled Community Slack channel](https://join.slack.com/t/coiled-users/shared_invite/zt-hx1fnr7k-In~Q8ui3XkQfvQon0yN5WQ) or by [tweeting](https://twitter.com/coiledhq) at us.